In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import glob

import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path


from keras.models import Sequential, Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
import cv2

from keras import backend as K



%reload_ext autoreload
%autoreload 2
%matplotlib inline




ModuleNotFoundError: No module named 'keras'

In [ ]:
data_dir = Path("data/chest_xray/chest_xray")
train_dir= data_dir/'train'
val_dir=data_dir/'val'
test_dir = data_dir / 'test'


A basic function to load train images from the directory and save it in a dataframe with labels.

In [ ]:
def load_train():
    normal_cases_dir = train_dir / 'NORMAL'
    pneumonia_cases_dir = train_dir / 'PNEUMONIA'

    # Get the list of all the images
    normal_cases = normal_cases_dir.glob('*.jpeg')
    pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')
    train_data=[]
    train_label=[]
    for img in normal_cases:
            train_data.append(img)
            train_label.append(0)
    for img in pneumonia_cases:
        train_data.append(img)
        train_label.append(1)
    df=pd.DataFrame(train_data)
    df.columns=['images']
    df['labels']=train_label
    df=df.sample(frac=1).reset_index(drop=True)
    return df

In [ ]:
train_data=load_train()
len(train_data)

### Loading and preprocessing validation and test data
Steps:-
* Load the image using imread()
* Since the images are of different length and widths, resize them to 224,224.
* Some images in our data are greyscale (1 channel) , therefore convert them to 3 channel
* Images using cv2 are read in BGR format(by default) , convert it to RGB.
* Normalize the image pixels by dividing by 255.

In [ ]:
def prepare_and_load(isval=True):
    if isval==True:
        normal_dir=val_dir/'NORMAL'
        pneumonia_dir=val_dir/'PNEUMONIA'
    else:
        normal_dir=test_dir/'NORMAL'
        pneumonia_dir=test_dir/'PNEUMONIA'
    normal_cases = normal_dir.glob('*.jpeg')
    pneumonia_cases = pneumonia_dir.glob('*.jpeg')
    data,labels=([] for x in range(2))
    def prepare(case):
        for img in case:
            img = cv2.imread(str(img))
            img = cv2.resize(img, (224,224))
            if img.shape[2] ==1:
                 img = np.dstack([img, img, img])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32)/255.
            if case==normal_cases:
                label = to_categorical(0, num_classes=2)
            else:
                label = to_categorical(1, num_classes=2)
            data.append(img)
            labels.append(label)
        return data,labels
    prepare(normal_cases)
    d,l=prepare(pneumonia_cases)
    d=np.array(d)
    l=np.array(l)
    return d,l
        

In [ ]:
val_data,val_labels=prepare_and_load(isval=True)
test_data,test_labels=prepare_and_load(isval=False)

### Generating batches of images for training
* It involves the same preprocessing steps as above, except that images are processed and returned in batches,defined by the batch size.
* We also use Image segmentation ( the slic function of skimage).Image segmentation is the process of partitioning a  image into multiple segments (sets of pixels, also known as super-pixels). The goal of segmentation is to simplify and/or change the representation of an image into something that is more meaningful and easier to analyze.
* The dataset we have is Imbalanced and has pneumonia cases three times the normal cases.The goal of our model is to optimize accuracy while training , which it can easily do by classifying most of the cases as infected(since the majority cases are infected, the model will have high accuracy), but it is biased aginst the underrepresented class.Thus we try to augment images of the normal class , by adding flipped , rotated and changing brightness of the original images.

In [ ]:
def data_gen(data, batch_size):
    # Get total number of samples in the data
    n = len(data)
    steps = n//batch_size
    
    # Define two numpy arrays for containing batch data and labels
    batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
    batch_labels = np.zeros((batch_size,2), dtype=np.float32)

    # Get a numpy array of all the indices of the input data
    indices = np.arange(n)
    
    # Initialize a counter
    i =0
    while True:
        np.random.shuffle(indices)
        # Get the next batch 
        count = 0
        next_batch = indices[(i*batch_size):(i+1)*batch_size]
        for j, idx in enumerate(next_batch):
            img_name = data.iloc[idx]['images']
            label = data.iloc[idx]['labels']
            
            # one hot encoding
            encoded_label = to_categorical(label, num_classes=2)
            # read the image and resize
            img = cv2.imread(str(img_name))
            img = cv2.resize(img, (224,224))
            
            # check if it's grayscale
            if img.shape[2]==1:
                img = np.dstack([img, img, img])
            
            # cv2 reads in BGR mode by default
            orig_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # normalize the image pixels
            orig_img = img.astype(np.float32)/255.
            
            #segmentation
            oig_img=slic(orig_img) 
            
            batch_data[count] = orig_img
            batch_labels[count] = encoded_label
            #augmentation
            seq = iaa.OneOf([
                 iaa.Fliplr(), # horizontal flips
                 iaa.Affine(rotate=20), # roatation
                 iaa.Multiply((1.2, 1.5))]) #random brightness
            # generating more samples of the undersampled class
            if label==0 and count < batch_size-2:
                aug_img1 = seq.augment_image(img)
                aug_img2 = seq.augment_image(img)
                aug_img1 = cv2.cvtColor(aug_img1, cv2.COLOR_BGR2RGB)
                aug_img2 = cv2.cvtColor(aug_img2, cv2.COLOR_BGR2RGB)
                aug_img1 = aug_img1.astype(np.float32)/255.
                aug_img2 = aug_img2.astype(np.float32)/255.

                batch_data[count+1] = aug_img1
                batch_labels[count+1] = encoded_label
                batch_data[count+2] = aug_img2
                batch_labels[count+2] = encoded_label
                count +=2
            
            else:
                count+=1
            
            if count==batch_size-1:
                break
            
        i+=1
        yield batch_data, batch_labels
            
        if i>=steps:
            i=0

In [ ]:
def vgg16_model( num_classes=None):

    model = VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))

    x=Dense(1024, activation='relu')(model.layers[-4].output)# add my own dense layer after the last conv block
    x=Dropout(0.7)(x)
    x=Dense(512,activation='relu')(x)
    x=Dropout(0.5)(x)
    x=Dense(2,activation='softmax')(x)
    model=Model(model.input,x)
    
    return model


In [ ]:
vgg_conv=vgg16_model(2)
for layer in vgg_conv.layers[:-10]:#freeze all layers except the last ten
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)


In [ ]:
vgg_conv.summary()

In [ ]:
opt = Adam(lr=0.0001, decay=1e-5)
early_stop = EarlyStopping(monitor='loss',patience=3,verbose=1)
vgg_conv.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

In [ ]:
batch_size = 16
nb_epochs = 5

# Get a train data generator
train_data_gen = data_gen(data=train_data, batch_size=batch_size)

# Define the number of training steps
nb_train_steps = train_data.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps, len(val_data)))

In [ ]:
# # Fit the model
history = vgg_conv.fit_generator(train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                              validation_data=(val_data,val_labels),callbacks=[early_stop],
                               class_weight={0:1.0, 1:0.4})

Let's see how it does on the test data.

In [ ]:
loss,acc=vgg_conv.evaluate(test_data,test_labels,batch_size=16)
print('Loss and accuracy',loss,'&',acc)

Since we are working on Imbalanced data, accuracy is not really a trustworthy measure.Let's view the classification report first.

In [ ]:
# Get predictions
pred = vgg_conv.predict(test_data, batch_size=16)
pred = np.argmax(pred, axis=-1)

# Original labels
labels = np.argmax(test_labels, axis=-1)
from sklearn.metrics import classification_report
print(classification_report(labels, pred))

**Precision** is a fraction of people actually having pneumonia to all those predicted by the model as having pneumonia. **Recall/Sensitivity** on the other hand refers to the fraction of people actually having pneumonia and are predicted positive by the model to the total number of people having pneumonia. Hence, it relates to the potential of a test to recognise subjects with the disease.
